# Pre-acknowledge about data set
- ExternalID=NaN # care home customers

# Setup

In [1]:
#Necessary import
import pandas as pd
import re

First, load the data, from the supplied data file

In [2]:
# Read in data
activity_dframe = pd.read_excel('CC Data 2020.xlsx', sheet_name=0)

In [3]:
# Have a look at the data set including size of it and general type of data
activity_dframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314766 entries, 0 to 314765
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Time         314766 non-null  object 
 1   VisitorID    314734 non-null  object 
 2   ExternalID   220993 non-null  float64
 3   Unnamed: 3   314734 non-null  object 
 4   Unnamed: 4   311515 non-null  object 
 5   LinkTitle    314734 non-null  object 
 6   LinkType     314734 non-null  object 
 7   ContentInfo  11575 non-null   object 
 8   Unnamed: 8   236 non-null     object 
 9   Unnamed: 9   30 non-null      object 
 10  Unnamed: 10  5 non-null       object 
 11  Unnamed: 11  5 non-null       object 
 12  Unnamed: 12  5 non-null       object 
 13  Unnamed: 13  5 non-null       object 
 14  Unnamed: 14  5 non-null       object 
 15  Unnamed: 15  5 non-null       object 
 16  Unnamed: 16  5 non-null       object 
dtypes: float64(1), object(16)
memory usage: 40.8+ MB


In [4]:
# Check the missing data
# For those columns missing much data, make sure whether to analyse and drop
activity_dframe.isnull().sum().sort_values(ascending=False)

Unnamed: 16    314761
Unnamed: 15    314761
Unnamed: 14    314761
Unnamed: 13    314761
Unnamed: 12    314761
Unnamed: 11    314761
Unnamed: 10    314761
Unnamed: 9     314736
Unnamed: 8     314530
ContentInfo    303191
ExternalID      93773
Unnamed: 4       3251
LinkType           32
LinkTitle          32
Unnamed: 3         32
VisitorID          32
Time                0
dtype: int64

In [26]:
activity_dframe.head()

,Time,VisitorID,ExternalID,Role,Building,LinkTitle,LinkType
0,2020-10-30 13:16:04.377000,9EDD833D-9D6B-46AC-A85E-DF21425162C4,8846.0,User,Night Support Service,Youtube Music,Internet
1,2020-10-30 13:14:46.100000,9EDD833D-9D6B-46AC-A85E-DF21425162C4,8846.0,User,Night Support Service,Youtube Music,Internet
2,2020-10-30 13:14:44.383000,9EDD833D-9D6B-46AC-A85E-DF21425162C4,8846.0,User,Night Support Service,My Music,Category
3,2020-10-30 13:14:38.740000,9EDD833D-9D6B-46AC-A85E-DF21425162C4,8846.0,User,Night Support Service,Entertainment,Category
4,2020-10-30 13:01:31.820000,679CBA7A-948B-4F2E-9432-002DDD5FFF8A,NaN,User,Broom Main Building,Forth 2,Radio


# Cleaning

The data had some some strange things in. Write down all of the strange things that you found here:

### I found the following strange things:
TODO: Rock Paper Scissors || Internet NULL
TODO: Mary Mungo &Midge || Internet NULL
* There are some additional columns from 'Unnamed: 8' to'Unnamed: 16' and strange value in the column 'Time'
* The incorrect data type
* Add names to columns 3 and 4
* Check the 'NaN'


# todo
- 'LinkTitle'='Mr. Robot HTML Test 2' & LinkType'='HTML page',
'ContentInfo' column will be seperated to next two rows
同一个 VisitorID(硬件)是否可能是不同的ExternalID
ExternalID=NaN care home customers

- 03/25 Lockdown



First, I am dropping column 'ContentInfo', columns right to it and strange rows whose 'Time' cells belong to the last row's column 'ContentInfo'

**The reason** is that the delimiter and line break of this tabular appear in the some value of 'ContentInfo' cells and
due to useless 'ContentInfo' column for project analysis, I am dropping the related columns and rows
- When the delimiter appears in the 'ContentInfo' column, the value of 'ContentInfo' was seperated to the extra columns right to that column
- The 'ContentInfo' is the last valid column, when the line break appears in this column, the value of 'ContentInfo' was seperated to next row's
first cell

In [7]:
# find the list of columns which should be dropped
_columns = activity_dframe.columns.values.tolist()
for _index, _element in enumerate(_columns):
    if 'ContentInfo' == _element:
        _columns = _columns[_index:]

# drop those columns
activity_dframe.drop(_columns, axis=1, inplace=True)

In [8]:
# find the strange values in 'Time' column which belong to last row
for _index, _row in activity_dframe.iterrows():
    try:
        _row['Time'] = pd.to_datetime(_row['Time'])
    except Exception as e:
        activity_dframe.drop(index=_index, inplace=True)
        activity_dframe.reset_index(drop=True, inplace=True)

In [9]:
activity_dframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314734 entries, 0 to 314733
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Time        314734 non-null  object 
 1   VisitorID   314714 non-null  object 
 2   ExternalID  220975 non-null  float64
 3   Unnamed: 3  314714 non-null  object 
 4   Unnamed: 4  311495 non-null  object 
 5   LinkTitle   314714 non-null  object 
 6   LinkType    314714 non-null  object 
dtypes: float64(1), object(6)
memory usage: 16.8+ MB


Second, I am adjusting the incorrect data type:
- Due to the less bit taken up and fewer precision problems, adjust column type related to ID to string

In [ ]:
# Adjust 'VisitorID' and 'ExternalID' to string
activity_dframe['VisitorID'] = activity_dframe['VisitorID'].astype('string')

# The NaN in the 'ExternalID' column represents care home customers and could be fill in '0'
activity_dframe['ExternalID'].fillna(0, inplace=True)
# Adjust 'VisitorID' to string
activity_dframe['ExternalID'] = activity_dframe['ExternalID'].astype('int').astype('string')

# Have a look at the description
activity_dframe.info()
activity_dframe.head()


### Third, add names to columns 3 and 4 according to the past data set.

In [10]:
activity_dframe.rename(columns={'Unnamed: 3': 'Role','Unnamed: 4':'Building'}, inplace=True)

### Forth, look into the 'Role' column
同一个 VisitorID(硬件)是否可能是不同的ExternalID

In [57]:
role_groupby = activity_dframe.groupby('Role')
role_groupby.count()

,Time,VisitorID,ExternalID,Building,LinkTitle,LinkType
Role,,,,,,
Employee,465,465,0,465,465,465
Support,639,639,366,330,639,639
User,313610,313610,220609,310700,313610,313610


In [65]:
print(role_groupby['VisitorID'].nunique())
# print(role_groupby['ExternalID'].nuique())
type(activity_dframe['ExternalID'][0])

Role
Employee      1
Support      10
User        610
Name: VisitorID, dtype: int64


numpy.float64

According to the above frame, there are three roles (i.e., Employee, Support, User), details as bellow:
- Some support and users have valid externalID, and others do not have that ID
- All of employees do not have ExternalID
- All of people have VisitorID

In [39]:
externalID_group_dict = activity_dframe.groupby('ExternalID').groups
externalID_group_dict.values()

# activity_dframe['VisitorID']==''

dict_values([Int64Index([ 15466,  15468,  15469,  15470,  15471,  15472,  15473, 116063,
            116065, 116066, 116067, 116068, 116069, 116071, 116072, 116077,
            116078, 116079, 116080, 116081, 116082, 116083, 116084, 125413,
            125414, 170869, 170870, 244244, 244245, 307344, 307345, 307346,
            307347, 307348, 307349, 307380, 307381],
           dtype='int64'), Int64Index([220688, 220689, 220690, 220691, 220692, 220693, 220694, 220695,
            220716, 220717, 229892, 229893, 229894, 247255],
           dtype='int64'), Int64Index([  6056,   6057,   6058,  21405,  28787,  28788,  28789,  28790,
             28791,  28792,
            ...
            266311, 266312, 266313, 266841, 289873, 289886, 293577, 307039,
            312947, 312948],
           dtype='int64', length=274), Int64Index([ 38501,  38502,  70588,  70590, 201386, 201387, 201388, 211989,
            216380, 217276, 217277, 217278, 264571, 269454, 269455, 269456,
            288040, 288

In [78]:
# Check the credibility of ExternalID

customer_dframe = pd.read_excel('CustomerDataAnon.xlsx', sheet_name=0)
customer_dframe.describe(include='all')

tt = customer_dframe['ID'].astype('float').astype('string')# int64
#activity_dframe['ExternalID'][0] # str

/Users/aishan/Projects/Pycharm/Notebook/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  after removing the cwd from sys.path.


In [25]:
activity_dframe['ExternalID'].fillna('0')

0         8846
1         8846
2         8846
3         8846
4            0
          ... 
314729    5107
314730    5107
314731    5107
314732    8639
314733    8639
Name: ExternalID, Length: 314734, dtype: object

### Usage:
- Time period to use platform
- Different activities in different time (afternoon, night, morning)

In [86]:
activity_dframe['Time'][0] - activity_dframe['Time'][1]


#TODO
# tmp = activity_dframe['ExternalID'].tolist()
# len(set(tmp))
# # len(set(tmp))
# # len(set(tt)-set(tmp))
# len(set(tmp)-set(tt)) #有一样的 ExternalID 有NaT 不好转化成别的如int 就把ID转成string了

datetime.timedelta(seconds=78, microseconds=277000)